In [133]:
# Load packages
import requests
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time


In [136]:
# Launch driver (DO NOT ATTEMPT!)
# Will get blocked
#for i in range(0,3):
#    i = i + 1
#    url = 'https://www.lazada.sg/watchessg/?from=wangpu&lang=en&page={}&q=All-Products&rating=4'.format(i)
#    driver = webdriver.Chrome()
#    driver.get(url)
#    time.sleep(15)

In [1]:
# Launch driver
def launch_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)

In [2]:
# Create soup
def create_soup(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    return soup

In [3]:
# Retrieve product names
def extract_names(soup):
    names = soup.find_all('div', {'class': 'c16H9d'})
    names_list = []
    for i in names:
        names_list.append(i.text)
    
    return names_list

In [4]:
# Retrieve product prices
def extract_prices(soup):
    prices = soup.find_all('span', {'class':'c13VH6'})
    prices_list = []
    for i in prices:
        prices_list.append(i.text)
    
    return prices_list

In [5]:
# Retrieve product ratings
def extract_ratings(soup):
    ratings = soup.find_all('div', {'class': 'c15YQ9'})
    ratings_list = []
    for i in ratings:
        ratings_list.append(str(i).count('<i class="c3dn4k c3EEAg">'))
    
    return ratings_list

In [6]:
# Store scraped data into a dataframe
def create_dataframe(names_list, prices_list, ratings_list):
    df = pd.DataFrame({'Name': names_list,
                      'Price': prices_list,
                      'Rating': ratings_list})
    
    return df

In [7]:
# Run entire pipeline
def full_pipeline(url):
        driver = webdriver.Chrome()
        driver.get(url)
        page_source = driver.page_source
        soup = create_soup(page_source)
        df = create_dataframe(extract_names(soup), extract_prices(soup), extract_ratings(soup))
        
        return df

In [173]:
# There are three pages for URL 'https://www.lazada.sg/watchessg/?from=wangpu&lang=en&q=All-Products&rating=4' .
# Hence, use below URLs for scraping to cover all three pages.
# Scrape these URLs separately and not in a loop to avoid your IP address from getting blocked.
i = 1 # After scraping i = 1, i.e. first page, proceed to scrape i = 2 (second page), i = 3 (third page).
url = 'https://www.lazada.sg/watchessg/?from=wangpu&lang=en&page={}&q=All-Products&rating=4'.format(i)

In [174]:
# Remove commented line of code below to run the full scraping pipeline
#if i == 1:
    df_page1 = full_pipeline(url)
elif i == 2:
    df_page2 = full_pipeline(url)
elif i == 3:
    df_page3 = full_pipeline(url)

In [179]:
# Concatenate all dataframes above
frames = [df_page1, df_page2, df_page3]
df_overall = pd.concat(frames, axis = 0).reset_index(drop = True)

In [181]:
# Export concatenated dataframe
df_overall.to_csv('webscraping_lazada_watchessg_21112020.csv', index = False)